In [1]:
from tqdm import tqdm
import pandas as pd
from transformers import pipeline


df_true = pd.read_csv("../data/labeled.csv")

df = df_true[df_true['class'] != 'unknown']
print(df)
print(df["class"].value_counts())

        Unnamed: 0 event_id_cnty       event_date  year  time_precision  \
0                0       BEL4179      23 May 2025  2025               1   
1                1       BEL4183      23 May 2025  2025               1   
2                2       BGR4378      23 May 2025  2025               1   
3                3       BGR4379      23 May 2025  2025               1   
4                4       BGR4380      23 May 2025  2025               1   
...            ...           ...              ...   ...             ...   
183058      183058          ROU2  08 January 2018  2018               1   
183059      183059       GRC2059  08 January 2018  2018               1   
183063      183063          HRV1  07 January 2018  2018               1   
183069      183069          BGR2  04 January 2018  2018               2   
183077      183077          BGR1  03 January 2018  2018               1   

         disorder_type event_type             sub_event_type  \
0       Demonstrations   Protests  

C:\Users\semvv\AppData\Local\Temp\ipykernel_10392\3073264609.py:6: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_true = pd.read_csv("../data/labeled.csv")


In [2]:
classifier = pipeline("zero-shot-classification", 
                      model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", 
                      device=0,
                      batch_size=8)

candidate_labels = [
    "women rights",
    "climate",
    "labor rights",
    "health care",
    "farmers",
    "environment",
    "public services",
    "palestine-israel conflict",
    "immigration",
    "unjust law enforcement",
    "ukraine-russia war",
    "discrimination",
    "education",
    "housing",
    "culture",
    "policies",
    "animal welfare",
    "pandemic",
    "lgbtq"
]

texts = df['notes'].tolist()

batch_size = 8
top_k = 3
predictions = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i + batch_size]
    results = classifier(batch, candidate_labels)

    if isinstance(results, dict):
        top_labels = results['labels'][:top_k]
        predictions.append(top_labels)
    else:
        for res in results:
            top_labels = res['labels'][:top_k]
            predictions.append(top_labels)

df['predicted_topics_top3'] = predictions

df.to_csv('../data/topics_zsc_top3.csv', index=False)

Device set to use cuda:0
100%|██████████| 15312/15312 [2:13:32<00:00,  1.91it/s]  
C:\Users\semvv\AppData\Local\Temp\ipykernel_10392\1545172000.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_topics_top3'] = predictions


In [6]:
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.io as pio
init_notebook_mode(True)

fig = px.bar(x=df["predicted_topic"].value_counts().index,y=df["predicted_topic"].value_counts(),color=df["predicted_topic"].value_counts().index,text=df["predicted_topic"].value_counts())
fig.update_traces(hovertemplate="Category:'%{x}' Counted: %{y}")
fig.update_layout(title={"text":"Category Counts","x":0.5,"font":{"size":35}},xaxis={"title":"Category","showgrid":False},yaxis={"title":"Value","showgrid":False},plot_bgcolor="white",width=800,height=500,showlegend=False)
iplot(fig)

KeyError: 'predicted_topic'

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_true = df['class']

y_pred_top3 = df['predicted_topics_top3']

top3_correct = [true_label in pred_list for true_label, pred_list in zip(y_true, y_pred_top3)]
top3_accuracy = sum(top3_correct) / len(top3_correct)

print("Top-3 Accuracy:", top3_accuracy)

Top-3 Accuracy: 0.855062288765164
